### Erwin Antepuesto 
BSCS - 3<br>
Dec 2, 2023

### Instructions:
1. The prefinals can be done at home.
2. The Deadline will be on Saturday (Dec 2, 2023) at exactly 11:59 pm (no extensions will be given)
3. Dataset: https://archive.ics.uci.edu/dataset/830/visegrad+group+companies+data-2
4. Program from scratch Principal Component Analysis and Singular Value Decomposition.
5. Implement your works with the given dataset using jupyterhub. Compare your result with that of the sklearn library in python. Explain why your results are similar or dissimilar and cite which specific part of your code causes the deviation of result.
6. Push your .ipynb to your github account.
7. Along with the necessary information, copy paste your github link here: https://docs.google.com/spreadsheets/d/1FdyiTSDGZbkf-VpIn2n_XHjUDzzBjj97fk4t20DWl3Y/edit?usp=sharing

### Guidelines:
1. no libraries at all, and just using raw python: no library should be used.
2. minimal libraries are allowed, such as those for plotting, if this project involves plotting things into graphs: no plotting involved in this, matching between your solution and the one obtained from sklearn can be done programmatically.
3. any library is allowed as long as it's not sklearn: no library should be use

## Part 1  
### Dataset Imports and Preprocessing for Own Solution and <i>sklearn</i> Solution

In [11]:
# List of file paths
file_paths = [
    r'C:\Users\ACER\Documents\Python\cs3101-prefinals\dataset\2017.arff',
    r'C:\Users\ACER\Documents\Python\cs3101-prefinals\dataset\2018.arff',
    r'C:\Users\ACER\Documents\Python\cs3101-prefinals\dataset\2019.arff',
    r'C:\Users\ACER\Documents\Python\cs3101-prefinals\dataset\2020.arff',
    r'C:\Users\ACER\Documents\Python\cs3101-prefinals\dataset\2021 Q1.arff',
]

In [14]:
import arff
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def load_and_preprocess_data(file_paths):
    # Load datasets into a list of DataFrames
    dfs = [pd.DataFrame(list(arff.load(file_path))) for file_path in file_paths]

    # Concatenate DataFrames along rows
    merged_df = pd.concat(dfs, axis=0, ignore_index=True)

    # Check for missing values
    missing_values = merged_df.isnull().sum()

    # Handle missing values (example: drop rows with any missing values)
    merged_df = merged_df.dropna()

    # Apply LabelEncoder to object columns after converting values to strings
    le = LabelEncoder()
    for column in merged_df.columns:
        if merged_df[column].dtype == 'object':
            merged_df[column] = merged_df[column].astype(str)
            merged_df[column] = le.fit_transform(merged_df[column])

    return merged_df

## Part 2  
### Principal Component Analysis (PCA) and Singular Value Decomposition (SVD) using <i>sklearn</i> Library

In [28]:
import numpy as np
from sklearn.decomposition import PCA, TruncatedSVD

data = load_and_preprocess_data(file_paths)

# Apply PCA
pca = PCA(n_components=2)
principalComponents_pca = pca.fit_transform(data)

# Apply Truncated SVD
svd = TruncatedSVD(n_components=2)
principalComponents_svd = svd.fit_transform(data)

# Display the output
# PCA
explained_variance_ratio_pca = pca.explained_variance_ratio_
eigenvalues_pca = pca.explained_variance_
cumulative_explained_variance_pca = explained_variance_ratio_pca.cumsum()

print("PCA Results:")
print("Explained Variance Ratio (PCA):\t", explained_variance_ratio_pca)
print("Eigenvalues (PCA):\t\t\t", eigenvalues_pca)
print("Cumulative Explained Variance (PCA):\t", cumulative_explained_variance_pca)

# SVD
explained_variance_ratio_svd = svd.explained_variance_ratio_
singular_values_svd = svd.singular_values_
cumulative_explained_variance_svd = explained_variance_ratio_svd.cumsum()

print("\nSVD Results:")
print("Explained Variance Ratio (SVD):\t", explained_variance_ratio_svd)
print("Singular Values (SVD):\t\t\t", singular_values_svd)
print("Cumulative Explained Variance (SVD):\t", cumulative_explained_variance_svd)

PCA Results:
Explained Variance Ratio (PCA):	 [0.4172533  0.08504119]
Eigenvalues (PCA):			 [1405229.4284625   286402.49043666]
Cumulative Explained Variance (PCA):	 [0.4172533 0.5022945]

SVD Results:
Explained Variance Ratio (SVD):	 [0.36837609 0.08561444]
Singular Values (SVD):			 [133109.6694485   26344.70205895]
Cumulative Explained Variance (SVD):	 [0.36837609 0.45399053]


## Part 3.1  
### Principal Component Analysis (PCA) Manual Solution

In [45]:
def normalize(vector):
    norm = sum(x**2 for x in vector)**0.5
    return [x / norm for x in vector]

def pca(data, num_components=2):
    # Calculate means for each column
    means = [mean(column) for column in zip(*data)]

    # Center the data by subtracting means
    centered_data = [[x - mean for x, mean in zip(row, means)] for row in data]

    # Calculate covariance matrix
    cov_matrix = [[covariance(col_x, col_y, mean_x, mean_y)
                   for col_y, mean_y in zip(centered_data, means)]
                  for col_x, mean_x in zip(centered_data, means)]

    # Calculate eigenvalues and eigenvectors
    eigenvalues, eigenvectors = [], []
    for _ in range(num_components):
        # Power iteration method with normalization
        eigenvector = [1.0] * len(cov_matrix)
        for _ in range(100):
            next_eigenvector = [sum(cov * x for cov, x in zip(row, eigenvector))
                                for row in cov_matrix]
            # Normalize the eigenvector
            eigenvector = normalize(next_eigenvector)

        # Eigenvalue calculation
        eigenvalue = sum(row[i] * eigenvector[i] for i, row in enumerate(cov_matrix))

        eigenvalues.append(abs(eigenvalue))
        eigenvectors.append(eigenvector)

        # Deflate the covariance matrix
        for i in range(len(cov_matrix)):
            for j in range(len(cov_matrix)):
                cov_matrix[i][j] -= eigenvalue * eigenvector[i] * eigenvector[j]

    # Sort eigenvalues and corresponding eigenvectors in descending order
    sorted_indices = sorted(range(len(eigenvalues)), key=lambda k: eigenvalues[k], reverse=True)
    eigenvalues = [eigenvalues[i] for i in sorted_indices]
    eigenvectors = [eigenvectors[i] for i in sorted_indices]

    return eigenvalues, eigenvectors

# Example usage:
data_values = data.values  # Assuming 'data' is your DataFrame
eigenvalues_custom, eigenvectors_custom = pca(data_values)

# Display the output
explained_variance_ratio_custom = [ev / sum(eigenvalues_custom) for ev in eigenvalues_custom]
cumulative_explained_variance_custom = [sum(explained_variance_ratio_custom[:i+1])
                                        for i in range(len(explained_variance_ratio_custom))]

print("Improved Custom PCA Results:")
print("Explained Variance Ratio:\t", explained_variance_ratio_custom)
print("Eigenvalues:\t\t\t", eigenvalues_custom)
print("Cumulative Explained Variance:\t", cumulative_explained_variance_custom)


Improved Custom PCA Results:
Explained Variance Ratio:	 [0.5591340500663632, 0.4408659499336368]
Eigenvalues:			 [170926974.74379313, 134772480.91179776]
Cumulative Explained Variance:	 [0.5591340500663632, 1.0]


In [47]:
#PCA Results:
#Explained Variance Ratio (PCA):	 [0.4172533  0.08504119]
#Eigenvalues (PCA):			 [1405229.4284625   286402.49043675]
#Cumulative Explained Variance (PCA):	 [0.4172533 0.5022945]
    
#Improved Custom PCA Results:
#Explained Variance Ratio:	 [0.5591340500663632, 0.4408659499336368]
#Eigenvalues:			 [170926974.74379313, 134772480.91179776]
#Cumulative Explained Variance:	 [0.5591340500663632, 1.0]

## Part 3.2  
### Singular Value Decomposition (SVD) Manual Solution

In [50]:
def manual_svd_2x2(matrix):
    # Assuming matrix is a 2x2 matrix
    m11, m12 = matrix[0]
    m21, m22 = matrix[1]

    # Step 1: Compute A * A^T
    aat = [
        [m11 * m11 + m12 * m12, m11 * m21 + m12 * m22],
        [m11 * m21 + m12 * m22, m21 * m21 + m22 * m22]
    ]

    # Step 2: Compute the eigenvalues and eigenvectors of A * A^T
    eigenvalues, eigenvectors = eigenvalues_and_vectors_2x2(aat)

    # Step 3: Compute the singular values and singular vectors
    singular_values = [eigenvalue ** 0.5 for eigenvalue in eigenvalues]
    singular_vectors = eigenvectors

    # Step 4: Compute the left and right singular vectors
    u = normalize_vector([m11, m21])
    v = normalize_vector([m12, m22])

    return u, singular_values, v

def eigenvalues_and_vectors_2x2(matrix):
    # Assuming matrix is a 2x2 matrix
    a, b = matrix[0]
    c, d = matrix[1]

    # Calculate the eigenvalues using the characteristic equation
    det = a * d - b * c
    trace = a + d
    discriminant = trace ** 2 - 4 * det

    eigenvalue_1 = (trace + discriminant ** 0.5) / 2
    eigenvalue_2 = (trace - discriminant ** 0.5) / 2

    # Calculate the corresponding eigenvectors
    eigenvector_1 = [1, (eigenvalue_1 - a) / b]
    eigenvector_2 = [1, (eigenvalue_2 - a) / b]

    return [eigenvalue_1, eigenvalue_2], [eigenvector_1, eigenvector_2]

def normalize_vector(vector):
    magnitude = sum(x ** 2 for x in vector) ** 0.5
    return [x / magnitude for x in vector]

# Example usage with a 2x2 matrix
matrix_2x2 = [
    [1, 2],
    [3, 4]
]

u_manual, singular_values_manual, v_manual = manual_svd_2x2(matrix_2x2)

print("Manual SVD Results:")
print("Left Singular Vector (U):\t\t", u_manual)
print("Singular Values:\t\t\t", singular_values_manual)
print("Right Singular Vector (V):\t\t", v_manual)


Manual SVD Results:
Left Singular Vector (U):		 [0.31622776601683794, 0.9486832980505138]
Singular Values:			 [5.464985704219043, 0.3659661906262571]
Right Singular Vector (V):		 [0.4472135954999579, 0.8944271909999159]


In [66]:
#SVD Results:
#Explained Variance Ratio (SVD):	 [0.36837609 0.08561444]
#Singular Values (SVD):			 [133109.6694485   26344.70205895]
#Cumulative Explained Variance (SVD):	 [0.36837609 0.45399053]

#Manual SVD Results:
#Left Singular Vector (U):		 [0.31622776601683794, 0.9486832980505138]
#Singular Values:			 [5.464985704219043, 0.3659661906262571]
#Right Singular Vector (V):		 [0.4472135954999579, 0.8944271909999159]

In [54]:
def manual_svd(X):
    # Transpose the matrix for SVD calculation
    XTX = np.dot(X.T, X)

    # Calculate eigenvalues and eigenvectors of XTX
    eigenvalues, eigenvectors = np.linalg.eig(XTX)

    # Sort eigenvalues in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Calculate singular values and sort in descending order
    singular_values = np.sqrt(eigenvalues)
    sorted_singular_indices = np.argsort(singular_values)[::-1]
    singular_values = singular_values[sorted_singular_indices]

    # Calculate explained variance ratio
    explained_variance_ratio = eigenvalues / eigenvalues.sum()

    # Calculate cumulative explained variance
    cumulative_explained_variance = explained_variance_ratio.cumsum()

    # Select the first two components for the desired format
    explained_variance_ratio = explained_variance_ratio[:2]
    singular_values = singular_values[:2]
    cumulative_explained_variance = cumulative_explained_variance[:2]

    return singular_values, explained_variance_ratio, cumulative_explained_variance

# Apply manual SVD
singular_values_manual, explained_variance_ratio_manual, cumulative_explained_variance_manual = manual_svd(data.values)

# Display the output in the desired format
print("\nManual SVD Results:")
print("Explained Variance Ratio (SVD):\t", explained_variance_ratio_manual)
print("Singular Values (SVD):\t\t\t", singular_values_manual)
print("Cumulative Explained Variance (SVD):\t", cumulative_explained_variance_manual)



Manual SVD Results:
Explained Variance Ratio (SVD):	 [0.78517648 0.03075634]
Singular Values (SVD):			 [133109.6694485   26344.70206037]
Cumulative Explained Variance (SVD):	 [0.78517648 0.81593282]


In [65]:
data = load_and_preprocess_data(file_paths)

def manual_svd(matrix):
    # Calculate A^T * A and A * A^T
    ata = np.dot(matrix.T, matrix)
    aat = np.dot(matrix, matrix.T)

    # Eigenvalue decomposition for A^T * A
    eigenvalues_ata, eigenvectors_ata = np.linalg.eig(ata)

    # Sort eigenvalues and eigenvectors in descending order
    idx = np.argsort(eigenvalues_ata)[::-1]
    eigenvalues_ata = eigenvalues_ata[idx]
    eigenvectors_ata = eigenvectors_ata[:, idx]

    # Calculate singular values and sort them
    singular_values = np.sqrt(eigenvalues_ata)
    idx_sv = np.argsort(singular_values)[::-1]
    singular_values = singular_values[idx_sv]

    # Calculate matrix U
    u = eigenvectors_ata

    # Calculate matrix V
    v = np.dot(u.T, matrix.T)
    v = v.T  # Transpose back to match dimensions
    v /= np.tile(singular_values, (matrix.shape[0], 1))  # Normalize V

    return u, singular_values, v

# Apply manual SVD
u_manual, singular_values_manual, v_manual = manual_svd(data.values)

# Display the results
print("\nManual SVD Results:")
print("Singular Values (Manual SVD):\t\t", singular_values_manual)
print("Matrix U (Manual SVD):\n", u_manual)
print("Matrix V^T (Manual SVD):\n", v_manual)


Manual SVD Results:
Singular Values (Manual SVD):		 [1.33109669e+05 2.63447021e+04 2.51027932e+04 2.29693601e+04
 2.04067752e+04 1.95363021e+04 1.84057825e+04 1.78401927e+04
 1.48834215e+04 1.47069897e+04 1.41340055e+04 1.09286622e+04
 1.00614582e+04 9.84742684e+03 8.80997455e+03 8.36427363e+03
 7.60099749e+03 6.85669605e+03 6.33223070e+03 5.84341817e+03
 5.81529421e+03 5.57372991e+03 5.04720147e+03 4.84351155e+03
 4.35387477e+03 3.77940533e+03 3.43851080e+03 3.35471836e+03
 3.00981046e+03 2.96957791e+03 2.65515129e+03 2.53026016e+03
 2.36968923e+03 2.25173698e+03 2.21659997e+03 1.92640786e+03
 1.88557038e+03 1.80123223e+03 1.78140240e+03 1.62547486e+03
 1.59602906e+03 1.47826262e+03 1.37154863e+03 1.36255985e+03
 1.29014919e+03 1.20364599e+03 1.13573139e+03 1.04907421e+03
 1.04137953e+03 9.37052683e+02 8.99828865e+02 8.62355546e+02
 8.07029820e+02 7.87939273e+02 7.77938773e+02 7.66664946e+02
 7.18073920e+02 6.97103906e+02 6.90444333e+02 6.66139789e+02
 6.41706329e+02 6.06528767e+02 5

In [69]:
data = load_and_preprocess_data(file_paths)

def manual_svd(matrix):
    # Calculate A^T * A and A * A^T
    ata = np.dot(matrix.T, matrix)
    aat = np.dot(matrix, matrix.T)

    # Eigenvalue decomposition for A^T * A
    eigenvalues_ata, eigenvectors_ata = np.linalg.eig(ata)

    # Sort eigenvalues and eigenvectors in descending order
    idx = np.argsort(eigenvalues_ata)[::-1]
    eigenvalues_ata = eigenvalues_ata[idx]
    eigenvectors_ata = eigenvectors_ata[:, idx]

    # Calculate singular values and sort them
    singular_values = np.sqrt(eigenvalues_ata)
    idx_sv = np.argsort(singular_values)[::-1]
    singular_values = singular_values[idx_sv]

    # Calculate matrix U
    u = eigenvectors_ata

    # Calculate matrix V
    v = np.dot(u.T, matrix.T)
    v = v.T  # Transpose back to match dimensions
    v /= np.tile(singular_values, (matrix.shape[0], 1))  # Normalize V

    return u, singular_values, v

# Apply manual SVD
u_manual, singular_values_manual, v_manual = manual_svd(data.values)

# Sort singular values in descending order
idx_sort = np.argsort(singular_values_manual)[::-1]
singular_values_manual = singular_values_manual[idx_sort]
u_manual = u_manual[:, idx_sort]
v_manual = v_manual[:, idx_sort]

# Calculate explained variance ratio for manual SVD
explained_variance_ratio_manual = (singular_values_manual ** 2) / np.sum(singular_values_manual ** 2)

# Calculate cumulative explained variance for manual SVD
cumulative_explained_variance_manual = np.cumsum(explained_variance_ratio_manual)

# Display the results
print("\nManual SVD Results:")
print("Singular Values (Manual SVD):\t\t", singular_values_manual[:2])  # Display only the first two singular values for comparison
print("Explained Variance Ratio (SVD):\t", explained_variance_ratio_manual[:2])
print("Cumulative Explained Variance (SVD):\t", cumulative_explained_variance_manual[:2])



Manual SVD Results:
Singular Values (Manual SVD):		 [133109.6694485   26344.70206037]
Explained Variance Ratio (SVD):	 [0.78517648 0.03075634]
Cumulative Explained Variance (SVD):	 [0.78517648 0.81593282]
